In [ ]:
import os
from urllib.parse import urlparse

domain = "https://www.grainger.com"
local_domain = urlparse(domain).netloc
# Create necessary directories if they don't exist
if not os.path.exists("text/"):
    os.mkdir("text/")
if not os.path.exists(f"text/{local_domain}/"):
    os.mkdir(f"text/{local_domain}/")
if not os.path.exists("processed"):
    os.mkdir("processed")

In [ ]:
# TEST 
import re

# Pattern to match product skus/codes
regex_pattern = re.compile(r'[A-Z0-9]{5,7}')
# Test strings
test_strings = [
    "1DKW3_1.pdf",
    "3VE59C-Operating-Instructions-and-Parts-Manual.pdf",
    "_3M-Disposable-Respirator-Dual-4JF99?opr=PDPBRDSP&analytics=dsbrItems_5ZZZ6.txt"
]

# Extract product codes from test strings
for test in test_strings:
    matches = regex_pattern.findall(test)
    print(f"Matches in '{test}': {matches}")


In [ ]:
# # PULL product codes from web scraped data and save as json
import os
import re
import json

# Define the regex pattern for product codes
regex_pattern = re.compile(r'[A-Z0-9]{5,7}')

# Directory containing the files
directory = 'text/www.grainger.com'

# Load existing product codes from the JSON file
existing_product_codes = []
print("os.path.exists('all_product_codes.json')", os.path.exists('all_product_codes.json'))
if os.path.exists('all_product_codes.json'):
    with open('all_product_codes.json', 'r') as f:
        existing_product_codes = json.load(f)

# List to store found product codes
product_codes = []

# Function to extract product codes from text
def extract_product_codes(text):
    return regex_pattern.findall(text)

# Iterate through all files in the directory
for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)

        # Check for product codes in the file name
        print(f"reading: {file_path}")
        codes_in_filename = extract_product_codes(file)
        product_codes.extend(codes_in_filename)

        # Check for product codes in the file content
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                codes_in_content = extract_product_codes(content)
                product_codes.extend(codes_in_content)
        except Exception as e:
            print(f"Could not read file {file_path}: {e}")

# Remove duplicates by converting the list to a set and back to a list
product_codes = list(set(product_codes))

# Combine existing and new product codes
product_codes = list(set(existing_product_codes + product_codes))

# Save all product codes to the JSON file
with open('all_product_codes.json', 'w') as f:
    json.dump(product_codes, f, indent=4)

print(f"Total product codes found: {len(product_codes)}")
print(f"Product codes saved in 'all_product_codes.json'")

In [ ]:
import os
import requests
import pandas as pd
import json
import html
import re

# Base URL and headers for the API
base_url = "https://mobile-rest-qa.nonprod.graingercloud.com/v1/product/detail"
headers = {
    "Content-Type": "application/json"
}

# Function to fetch and process data
def fetch_product_details(skus):
    params = {
        "partNumbers": skus,
        "extraInfo": "false"
    }
    print(f"Fetching details for SKUs: {skus}")  # Debugging print statement
    response = requests.get(base_url, headers=headers, params=params, verify=False)
    if response.status_code == 200:
        try:
            data = response.json()
            results = []
            for item in data:
                brand = item.get("brand", {}).get("name", "N/A")
                code = item.get("code", "N/A")
                name = item.get("name", "N/A")
                picture_url = item.get("pictureUrl600", "N/A")
                price = item.get("priceData", {}).get("formattedPrice", "N/A")
                description = item.get("productDetailsDescription", "N/A")

                results.append({
                    "Brand": brand,
                    "Code": code,
                    "Name": name,
                    "PictureUrl600": picture_url,
                    "Price": price,
                    "Description": description
                })

            print(f"Successfully processed {len(results)} items.")  # Debugging print statement
            return pd.DataFrame(results) if results else None
        except Exception as e:
            print(f"Error parsing response for {skus}: {e}")
            return None
    else:
        print(f"Failed to fetch details for {skus}: Status code {response.status_code}")
        return None

# Load the product codes from the JSON file
try:
    with open('processed/all_product_codes.json', 'r') as f:
        product_codes = json.load(f)
except FileNotFoundError:
    print("JSON file 'all_product_codes.json' not found.")
else:
    print(f"Total product codes found: {len(product_codes)}")

# Product codes in chunks of 100
chunk_size = 1 # Using 1 to prevent errors with invalid product codes preventing the rest from loading.
chunks = [product_codes[i:i + chunk_size] for i in range(0, len(product_codes), chunk_size)]

# Initialize DataFrame
df = pd.DataFrame(columns=["Brand", "Code", "Name", "PictureUrl600", "Price", "Description"])

# Load existing data if the file exists
parquet_path = 'processed/grainger_products.parquet'

if os.path.exists(parquet_path):
    df_existing = pd.read_parquet(parquet_path)
    df = pd.concat([df, df_existing], ignore_index=True)
    print("Loaded existing data from 'modules/vector_index/processed/grainger_products.parquet'.")

# Fetch and append new data
failed_chunks = []
for chunk in chunks:
    try:
        details = fetch_product_details(chunk)
        if details is not None:
            df = pd.concat([df, details], ignore_index=True)
            print(f"Successfully appended {len(details)} rows to DataFrame.")  # Debugging print statement
        else:
            print(f"No details fetched for chunk: {chunk}. Removing from source.")
            failed_chunks.extend(chunk)
    except Exception as e:
        print(f"Failed to fetch details for chunk: {chunk}, Error: {e}")
        failed_chunks.extend(chunk)

# Ensure all column names are strings
df.columns = df.columns.astype(str)

# Complete data cleaning
df.fillna("", inplace=True)
df.drop_duplicates(inplace=True)

def clean_code(code):
    # Extract the part before any space or other characters
    return re.split(r'\s|[-_()]+', code, 1)[0]

df['Code'] = df['Code'].apply(clean_code)
# Remove HTML characters from all columns
df = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)

# Save to Parquet
parquet_path = 'processed/grainger_products.parquet'
os.makedirs('processed', exist_ok=True)
df.to_parquet(parquet_path, index=False)
print(f"Product details have been saved to {parquet_path}")
print("\nHead of DataFrame:")
print(df.head(), "\n")
print("Tail of DataFrame:")
print(df.tail(), "\n") 
print("Size of DataFrame:", df.size, "\n")
print("Values in DataFrame:")
print(df.values)


In [ ]:

# Save to Parquet
parquet_path = 'processed/grainger_products.parquet'
os.makedirs('processed', exist_ok=True)
df.to_parquet(parquet_path, index=False)
print(f"Product details have been saved to {parquet_path}")
print("\nHead of DataFrame:")
print(df.head(), "\n")
print("Tail of DataFrame:")
print(df.tail(), "\n") 
print("Size of DataFrame:", df.size, "\n")
print("Values in DataFrame:")
print(df.values)

In [ ]:
# Complete data cleaning
df.fillna("", inplace=True)
df.drop_duplicates(inplace=True)

def clean_code(code):
    # Extract the part before any space (if exists)
    return re.split(r'\s+', code, 1)[0]

df['Code'] = df['Code'].apply(clean_code)
# Remove HTML characters from all columns
df = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)



In [ ]:

# Save to Parquet
os.makedirs('processed', exist_ok=True)
df.to_parquet('processed/grainger_products.parquet', index=False)
print("Product details have been saved to 'processed/grainger_products.parquet'")
print("\nHead of DataFrame:")
print(df.head(), "\n")
print("Tail of DataFrame:")
print(df.tail(), "\n")
print("Size of DataFrame:", df.size, "\n")
print("Values in DataFrame:")
print(df.values)


In [ ]:
import pandas as pd
df = pd.read_parquet('processed/grainger_products.parquet')
df.head()